# Exploring the NIFTY 50 index under different SMA regimes

Markets tend to be manic depressive, going from excessive optimism to excessive pessimism. The distribution of returns tend to be different based on the "regime." A simple way to demarcate these regimes is by using an SMA (Simple Moving Average) to bucket returns into binary groups. Here, we explore how returns are distributed in these buckets.

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(RPostgres)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)
library(reshape2)
library(ggrepel)
library(lubridate)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
indices<-Indices()

In [ ]:
indexName <- 'NIFTY 50'
smaLb <- 50 #number of days
px <- indices$NseTimeSeries() %>%
    filter(NAME == indexName) %>%
    select(TIME_STAMP, CLOSE) %>%
    collect()

#if we break by years, make sure that they have at least 200 days
numDays <- px %>%
    group_by(Y = year(TIME_STAMP)) %>%
    summarize(N = n())

yrs <- numDays %>%
    filter(N >= 200) %>%
    pull(Y)

In [ ]:
pXts <- xts(px$CLOSE, px$TIME_STAMP)
names(pXts) <- c('INDEX')
pXts$SMA <- SMA(pXts[,1], smaLb)
pXts$D_RET <- dailyReturn(pXts$INDEX)
pXts$D_RET_PCT <- pXts$D_RET * 100

pXts <- na.omit(pXts)

In [ ]:
statByYear <- data.frame(Y = 0, N_ABOVE = 0, N_BELOW = 0, AVG_ABOVE = 0.0, AVG_BELOW = 0.0, SD_ABOVE = 0.0, SD_BELOW = 0.0)

for(yr in yrs){
    yrStr <- toString(yr)
    numAbove <- as.numeric(sum(ifelse(pXts[yrStr,'INDEX'] > pXts[yrStr,'SMA'], 1, 0)))
    numBelow <- as.numeric(sum(ifelse(pXts[yrStr,'INDEX'] <= pXts[yrStr,'SMA'], 1, 0)))
    
    avgAbove <- as.numeric(mean(pXts[pXts[yrStr,'INDEX'] > pXts[yrStr,'SMA'], 'D_RET_PCT']))
    avgBelow <- as.numeric(mean(pXts[pXts[yrStr,'INDEX'] <= pXts[yrStr,'SMA'], 'D_RET_PCT']))
    
    sdAbove <- as.numeric(sd(pXts[pXts[yrStr,'INDEX'] > pXts[yrStr,'SMA'], 'D_RET_PCT']))
    sdBelow <- as.numeric(sd(pXts[pXts[yrStr,'INDEX'] <= pXts[yrStr,'SMA'], 'D_RET_PCT']))
    
    statByYear <- rbind(statByYear, c(yr, numAbove, numBelow, avgAbove, avgBelow, sdAbove, sdBelow))
}

statByYear <- statByYear[-1,]
print(statByYear)

The NIFTY got hammered by the GFC in 2008, only to be reflated in 2009. The years prior to do that was a different "regime." Not sure if they can be used in back-tests any more. Let's look at the data from 2010 onwards.

In [ ]:
rollN <- 200
stat200 <- rollapply(pXts["2010/",], rollN, function(X){
    numAbove <- as.numeric(sum(ifelse(X[,'INDEX'] > X[,'SMA'], 1, 0)))
    numBelow <- as.numeric(sum(ifelse(X[,'INDEX'] <= X[,'SMA'], 1, 0)))
    
    avgAbove <- as.numeric(mean(X[X[,'INDEX'] > X[,'SMA'], 'D_RET_PCT']))
    avgBelow <- as.numeric(mean(X[X[,'INDEX'] <= X[,'SMA'], 'D_RET_PCT']))
    
    sdAbove <- as.numeric(sd(X[X[,'INDEX'] > X[,'SMA'], 'D_RET_PCT']))
    sdBelow <- as.numeric(sd(X[X[,'INDEX'] <= X[,'SMA'], 'D_RET_PCT']))
    
    xts(matrix(c(numAbove, numBelow, avgAbove, avgBelow, sdAbove, sdBelow), nrow=1), as.Date(last(index(X))))
}, by.column = FALSE)

stat200 <- na.omit(stat200)
names(stat200) <- c('N_ABOVE', 'N_BELOW', 'AVG_ABOVE', 'AVG_BELOW', 'SD_ABOVE', 'SD_BELOW')

print("stat200: HEAD")
print(head(stat200))

print("stat200: TAIL")
print(tail(stat200))

In [ ]:
stat200Df <- data.frame(stat200)
stat200Df$T <- as.Date(index(stat200))

plotStart <- first(index(stat200))
plotEnd <- last(index(stat200))
xAxisTicks<-seq(plotStart, plotEnd, length.out=10)

ggplot(stat200Df, aes(x=T)) +
    theme_economist() +
    geom_line(aes(y=AVG_ABOVE, color='a')) +
    geom_ribbon(aes(ymin = AVG_ABOVE - SD_ABOVE, ymax = AVG_ABOVE + SD_ABOVE), fill='springgreen', alpha=0.3) +
    geom_line(aes(y=AVG_BELOW, color='b')) +
    geom_ribbon(aes(ymin = AVG_BELOW - SD_BELOW, ymax = AVG_BELOW + SD_BELOW), fill='darkorange', alpha=0.3) +
    scale_x_date(breaks = xAxisTicks) +
    labs(x = "", y="returns(%)", fill="", color="", 
         title = sprintf("%s %d-day Avg. Daily Returns", indexName, rollN),
         subtitle = sprintf("%d-SMA [%s:%s]", smaLb, plotStart, plotEnd)) +
    scale_colour_manual(name = '', values =c('a'='olivedrab','b'='maroon'), labels = c('above','below')) +
    annotate("text", x=plotEnd, y=min(stat200Df$AVG_ABOVE - stat200Df$SD_ABOVE), label = "@StockViz", 
             hjust=1.1, vjust=0, col="white", cex=6, fontface = "bold", alpha = 0.8)


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)